In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [4]:
try:
    hillsborough_plant_df = pd.read_pickle('Florida_Native_Flowers.pkl')
    scrape = False
except:
    scrape = True

In [5]:
if scrape:
    URL = 'https://fnps.org/plants/specieslist/Hillsborough'
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
#     print(soup.prettify())
    soup_links  = soup.find_all('a')
    hillsborough_plants_links  = [x for x in soup_links if 'https://fnps.org/plants/plant/' in str(x)]
    hillsborough_plants_dict = {x.text:x['href'] for x in hillsborough_plants_links}

In [6]:
def to_dict(dictionary, string):
    return dictionary.update({string.split(':')[0]:string.split(':')[1].strip()})
def scrape_plant_info(soup2):
    plant_name = soup2.find(class_='fittest1').text
    plant_aka = soup2.find(id='subtitles').find('h2').text
    plant_scientific = soup2.find(id='subtitles').find('h3').text

    wildlife_list = [re.search('icons/(.*)-',x['src']).group(1) for x in soup2.find(id='wildlife_icons').find_all('img')]
    light_requirements = [re.search('icons/(.*)-',x['src']).group(1) for x in soup2.find_all(alt='light requirement')]
            
    plant_info_dict = {'plant_name': plant_name,
                  'plant_aka':plant_aka,
                  'plant_scientific': plant_scientific,
                  'wildlife_list': wildlife_list,
                      'plant_link':link,
                      'light_requirements':light_requirements}

    plant_info = [x.text for x in soup2.find_all(class_='plantdiv')]

    for x in plant_info:
        if 'Size:' in x:
            x = x.replace('Size: ', '').split('\xa0\xa0\xa0')
            to_dict(plant_info_dict, x[0])
            if len(x)>1:
                to_dict(plant_info_dict, x[1])
        elif 'Light: ' in x:
            pass
        else:
            try:
                to_dict(plant_info_dict, x)
            except IndexError:
                plant_info_dict.update({'Wildlife Notes': x})

    suitable_zones = soup2.find(id='zonemap').find('span').text
    plant_info_dict['suitable_zones'] = suitable_zones
    return plant_info_dict

In [7]:
page_dicts = []
if scrape:
    for link in list(hillsborough_plants_dict.values()):
        print(link)
        page2 = requests.get(link)
        soup2 = BeautifulSoup(page2.content, 'html.parser')
        plant_info_dict = scrape_plant_info(soup2)
        page_dicts.append(plant_info_dict)
        
    hillsborough_plant_df = pd.DataFrame.from_records(page_dicts)
    hillsborough_plant_df.to_pickle('Hillsborough_Native_Plants.pkl')
    hillsborough_plant_df.to_csv('Hillsborough_Native_Plants.csv')
hillsborough_plant_df.head()

,Availability,Considerations,Ethnobotany,Flower Color,Form,Fruit Color,Height,Life Span,Light,Moisture Tolerance,...,Width,Wildlife,Wildlife Notes,light_requirements,plant_aka,plant_link,plant_name,plant_scientific,suitable_zones,wildlife_list
0,"Native nurseries, FNPS plant sales, Friends","Fast growing, fairly weak hence somewhat prone...",NaN,"yellow,NA",tree,"green,brown",30-70 ft,long-lived perennial,,,...,20-40 ft,,"Attracts pollinators, particularly native bees...","[sun, part-shade]","box-elder, ash-leaf maple",https://fnps.org/plants/plant/acer-negundo,Acer negundo,Sapindaceae,"8A,8B,9A","[birds, animals, butterfly]"
1,"Big box stores, Quality nurseries, Native nurs...","Fast growing, fairly weak. Shallow, wide-sprea...",Sap once used as a source of sugar. Wood stri...,red,tree,red,45 to 50 ft,long-lived perennial,,,...,35 to 50 ft,,Birds and other wildlife consume seeds.\nLarva...,"[sun, part-shade]",red maple,https://fnps.org/plants/plant/acer-rubrum,Acer rubrum,Sapindaceae,"8A,8B,9A,9B,10A,10B","[birds, animals]"
2,Native nurseries,Difficult to establish because of long taproot.,Edible fruit.,"red,purple",tree,"green,brown,NA",to 15 ft,long-lived perennial,,,...,to 10 ft,,Larval host for zebra swallowtail (Eurytides m...,[part-shade],smallflower pawpaw,https://fnps.org/plants/plant/asimina-parviflora,Asimina parviflora,Annonaceae,,"[animals, caterpillar]"
3,Native nurseries,NaN,NaN,white,tree,green,15 to 40 (60) ft,long-lived perennial,,,...,10 to 30 ft,,Larval host for mangrove buckeye (Junonia eva...,[sun],black mangrove,https://fnps.org/plants/plant/avicennia-germinans,Avicennia germinans,Avicenniaceae,"9A,9B,10A,10B,11",[caterpillar]
4,"Quality nurseries, Native nurseries, FNPS plan...",NaN,NaN,"white,green,NA",tree,"brown,NA",20-60 ft,long-lived perennial,,,...,20-50 ft,,Larval host for dingy purplewing (Eunica moni...,"[sun, part-shade]",gumbo-limbo,https://fnps.org/plants/plant/bursera-simaruba,Bursera simaruba,Burseraceae,"9B,10A,10B,11","[birds, caterpillar]"


In [8]:
# link = list(hillsborough_plants_dict.values())[5]
# page2 = requests.get(link)
# soup2 = BeautifulSoup(page2.content, 'html.parser')

# print(soup2.prettify())

In [88]:
def height_cleanup(cell):
    cell = cell.replace('climbs to 10 or more.','10 ft').replace('spreading','').replace('60(40) -75 ft','60-75 ft')\
                .replace(".5' to 1.5'",'.5-1.5 ft').replace('5-10.','5-10 ft')
    new_cell = ' '.join(cell.lower().split(' ')[0:-1])
    delete_str_list = ['variable, mostly ', 'most ',
                      '(mostly 5)', 'climbing', '(5 in flower)',
                      '3 rarely', 'with support, ']
    for string in delete_str_list:
        new_cell = new_cell.replace(string, '')
    new_cell = new_cell.replace(' to ', '-')\
                .replace('to ', '')\
                .replace(' - ', '-')\
                  .split(' (')[0]\
                .split('-')+[cell.split(' ')[-1]]
    return new_cell

def fix_height(col):
    new_col = col.fillna(' ').replace('climbing','').apply(lambda x: height_cleanup(x))\
                                            .apply(lambda x: [float(y) if y not in ['','inches','ft','feet'] else y for y in x])\
                                            .apply(lambda x: [x[0]]+x if len(x)<3 else x)\
                                            .apply(lambda x: [x[0]/12,x[1]/12] if x[2]=='inches' else x[0:2])
    return list(new_col)

In [10]:
hillsborough_plant_df.Height =  fix_height(hillsborough_plant_df.Height)
hillsborough_plant_df[['height_min_ft','height_max_ft']] = pd.DataFrame(hillsborough_plant_df.Height.values.tolist())
hillsborough_plant_df.head()

,Availability,Considerations,Ethnobotany,Flower Color,Form,Fruit Color,Height,Life Span,Light,Moisture Tolerance,...,Wildlife Notes,light_requirements,plant_aka,plant_link,plant_name,plant_scientific,suitable_zones,wildlife_list,height_min_ft,height_max_ft
0,"Native nurseries, FNPS plant sales, Friends","Fast growing, fairly weak hence somewhat prone...",NaN,"yellow,NA",tree,"green,brown","[30.0, 70.0]",long-lived perennial,,,...,"Attracts pollinators, particularly native bees...","[sun, part-shade]","box-elder, ash-leaf maple",https://fnps.org/plants/plant/acer-negundo,Acer negundo,Sapindaceae,"8A,8B,9A","[birds, animals, butterfly]",30,70
1,"Big box stores, Quality nurseries, Native nurs...","Fast growing, fairly weak. Shallow, wide-sprea...",Sap once used as a source of sugar. Wood stri...,red,tree,red,"[45.0, 50.0]",long-lived perennial,,,...,Birds and other wildlife consume seeds.\nLarva...,"[sun, part-shade]",red maple,https://fnps.org/plants/plant/acer-rubrum,Acer rubrum,Sapindaceae,"8A,8B,9A,9B,10A,10B","[birds, animals]",45,50
2,Native nurseries,Difficult to establish because of long taproot.,Edible fruit.,"red,purple",tree,"green,brown,NA","[15.0, 15.0]",long-lived perennial,,,...,Larval host for zebra swallowtail (Eurytides m...,[part-shade],smallflower pawpaw,https://fnps.org/plants/plant/asimina-parviflora,Asimina parviflora,Annonaceae,,"[animals, caterpillar]",15,15
3,Native nurseries,NaN,NaN,white,tree,green,"[15.0, 40.0]",long-lived perennial,,,...,Larval host for mangrove buckeye (Junonia eva...,[sun],black mangrove,https://fnps.org/plants/plant/avicennia-germinans,Avicennia germinans,Avicenniaceae,"9A,9B,10A,10B,11",[caterpillar],15,40
4,"Quality nurseries, Native nurseries, FNPS plan...",NaN,NaN,"white,green,NA",tree,"brown,NA","[20.0, 60.0]",long-lived perennial,,,...,Larval host for dingy purplewing (Eunica moni...,"[sun, part-shade]",gumbo-limbo,https://fnps.org/plants/plant/bursera-simaruba,Bursera simaruba,Burseraceae,"9B,10A,10B,11","[birds, caterpillar]",20,60


In [11]:
hillsborough_plant_df['9B_suitable'] = hillsborough_plant_df.suitable_zones.apply(lambda x: '9B' in str(x))
hillsborough_plant_df['9B_suitable'].head()

0    False
1     True
2    False
3     True
4     True
Name: 9B_suitable, dtype: bool

In [12]:
hillsborough_plant_df = hillsborough_plant_df[hillsborough_plant_df['9B_suitable']]
hillsborough_plant_df.shape

(287, 33)

In [13]:
butterfly = hillsborough_plant_df[hillsborough_plant_df.wildlife_list.apply(lambda x: 'butterfly' in x)]
butterfly.shape

(167, 33)

In [15]:
# filter for full-sun
# filter for sand soil
# filter for not deciduous

sun_sand_evergreen = butterfly[butterfly.light_requirements.apply(lambda x: 'sun' in x)]
sun_sand_evergreen = sun_sand_evergreen[sun_sand_evergreen['Soil or other substrate'].fillna('').apply(lambda x: 'sand' in x.lower())]
sun_sand_evergreen = sun_sand_evergreen[sun_sand_evergreen.Phenology.fillna('').apply(lambda x: 'deciduous' not in x.lower())]
sun_sand_evergreen.shape

(119, 33)

In [16]:
big_bushes = sun_sand_evergreen[sun_sand_evergreen.height_min_ft.replace('',6)>5]
big_bushes = big_bushes[big_bushes.height_max_ft.replace('',15)<=15]
big_bushes.shape

(18, 33)

In [87]:
coi = ['plant_name','plant_aka','Form','height_min_ft','height_max_ft',
            'Life Span','Soil pH Range','Width','light_requirements',
            'wildlife_list','Flower Color','Considerations','Recommended Uses',
            'Other Comments','Noted for','Wildlife Notes','plant_link','plant_scientific']

### All Florida

In [24]:
all_florida_native_plants_list_link = 'https://fnps.org/plants/browse'
scrape_all_florida_plants = True
if scrape_all_florida_plants:
    page = requests.get(all_florida_native_plants_list_link)

    soup = BeautifulSoup(page.content, 'html.parser')
#     print(soup.prettify())
    soup_links  = soup.find_all('a')
    florida_plants_links  = [x for x in soup_links if '/plants/plant/' in str(x)]
    florida_plants_dict = {x.text:'https://fnps.org'+x['href'] for x in florida_plants_links}

florida_plants_dict

{'Acacia choriophylla': 'https://fnps.org/plants/plant/acacia-choriophylla',
 'Acer negundo': 'https://fnps.org/plants/plant/acer-negundo',
 'Acer rubrum': 'https://fnps.org/plants/plant/acer-rubrum',
 'Acer saccharinum': 'https://fnps.org/plants/plant/acer-saccharinum',
 'Acer saccharum subsp. floridanum': 'https://fnps.org/plants/plant/acer-saccharum-subsp-floridanum',
 'Achillea millefolium': 'https://fnps.org/plants/plant/achillea-millefolium',
 'Acoelorraphe wrightii': 'https://fnps.org/plants/plant/acoelorraphe-wrightii',
 'Acrostichum danaeifolium': 'https://fnps.org/plants/plant/acrostichum-danaeifolium',
 'Adiantum capillus-veneris': 'https://fnps.org/plants/plant/adiantum-capillus-veneris',
 'Aesculus pavia': 'https://fnps.org/plants/plant/aesculus-pavia',
 'Agalinis fasciculata': 'https://fnps.org/plants/plant/agalinis-fasciculata',
 'Agarista populifolia': 'https://fnps.org/plants/plant/agarista-populifolia',
 'Agave decipiens': 'https://fnps.org/plants/plant/agave-decipien

In [27]:
page_dicts = []
if scrape_all_florida_plants:
    for link in list(florida_plants_dict.values()):
        print(link)
        page2 = requests.get(link)
        soup2 = BeautifulSoup(page2.content, 'html.parser')
        plant_info_dict = scrape_plant_info(soup2)
        page_dicts.append(plant_info_dict)
        
    florida_plant_df = pd.DataFrame.from_records(page_dicts)
    florida_plant_df.to_pickle('Florida_Native_Plants.pkl')
    florida_plant_df.to_csv('Florida_Native_Plants.csv')
florida_plant_df.head()

https://fnps.org/plants/plant/acacia-choriophylla
https://fnps.org/plants/plant/acer-negundo
https://fnps.org/plants/plant/acer-rubrum
https://fnps.org/plants/plant/acer-saccharinum
https://fnps.org/plants/plant/acer-saccharum-subsp-floridanum
https://fnps.org/plants/plant/achillea-millefolium
https://fnps.org/plants/plant/acoelorraphe-wrightii
https://fnps.org/plants/plant/acrostichum-danaeifolium
https://fnps.org/plants/plant/adiantum-capillus-veneris
https://fnps.org/plants/plant/aesculus-pavia
https://fnps.org/plants/plant/agalinis-fasciculata
https://fnps.org/plants/plant/agarista-populifolia
https://fnps.org/plants/plant/agave-decipiens
https://fnps.org/plants/plant/aletris-lutea
https://fnps.org/plants/plant/allium-canadense
https://fnps.org/plants/plant/alnus-serrulata
https://fnps.org/plants/plant/alvaradoa-amorphoides
https://fnps.org/plants/plant/ambrosia-hispida
https://fnps.org/plants/plant/amelanchier-arborea
https://fnps.org/plants/plant/amorpha-fruticosa
https://fnps.or

https://fnps.org/plants/plant/crataegus-flava
https://fnps.org/plants/plant/crataegus-marshallii
https://fnps.org/plants/plant/crataegus-michauxii
https://fnps.org/plants/plant/crataegus-phaenopyrum
https://fnps.org/plants/plant/crinum-americanum
https://fnps.org/plants/plant/crossopetalum-ilicifolium
https://fnps.org/plants/plant/crossopetalum-rhacoma
https://fnps.org/plants/plant/ctenium-aromaticum
https://fnps.org/plants/plant/cynophalla-flexuosa
https://fnps.org/plants/plant/cyrilla-racemiflora
https://fnps.org/plants/plant/dalea-carnea
https://fnps.org/plants/plant/dalea-feayi
https://fnps.org/plants/plant/dalea-pinnata
https://fnps.org/plants/plant/decumaria-barbara
https://fnps.org/plants/plant/dicerandra-spp
https://fnps.org/plants/plant/diospyros-virginiana
https://fnps.org/plants/plant/dirca-palustris
https://fnps.org/plants/plant/distichlis-spicata
https://fnps.org/plants/plant/dodonaea-viscosa
https://fnps.org/plants/plant/dryopteris-ludoviciana
https://fnps.org/plants/plan

https://fnps.org/plants/plant/leucothrinax-morrisii
https://fnps.org/plants/plant/liatris-elegans
https://fnps.org/plants/plant/liatris-gracilis
https://fnps.org/plants/plant/liatris-ohlingerae
https://fnps.org/plants/plant/liatris-spicata
https://fnps.org/plants/plant/liatris-spp
https://fnps.org/plants/plant/liatris-spp-dry-site-species
https://fnps.org/plants/plant/liatris-tenuifolia
https://fnps.org/plants/plant/lilaeopsis-carolinensis
https://fnps.org/plants/plant/lilium-catesbaei
https://fnps.org/plants/plant/linaria-canadensis
https://fnps.org/plants/plant/lindera-benzoin
https://fnps.org/plants/plant/lippia-stoechadifolia
https://fnps.org/plants/plant/liquidambar-styraciflua
https://fnps.org/plants/plant/liriodendron-tulipifera
https://fnps.org/plants/plant/litsea-aestivalis
https://fnps.org/plants/plant/lobelia-cardinalis
https://fnps.org/plants/plant/lonicera-sempervirens
https://fnps.org/plants/plant/ludwigia-repens
https://fnps.org/plants/plant/lupinus-diffusus
https://fnps

https://fnps.org/plants/plant/quercus-phellos
https://fnps.org/plants/plant/quercus-shumardii
https://fnps.org/plants/plant/quercus-stellata
https://fnps.org/plants/plant/quercus-velutina
https://fnps.org/plants/plant/quercus-virginiana
https://fnps.org/plants/plant/randia-aculeata
https://fnps.org/plants/plant/ratibida-pinnata
https://fnps.org/plants/plant/reynosia-septentrionalis
https://fnps.org/plants/plant/rhabdadenia-biflora
https://fnps.org/plants/plant/rhamnus-caroliniana
https://fnps.org/plants/plant/rhapidophyllum-hystrix
https://fnps.org/plants/plant/rhexia-spp
https://fnps.org/plants/plant/rhipsalis-baccifera
https://fnps.org/plants/plant/rhizophora-mangle
https://fnps.org/plants/plant/rhododendron-austrinum
https://fnps.org/plants/plant/rhododendron-canescens
https://fnps.org/plants/plant/rhododendron-minus-var-chapmanii
https://fnps.org/plants/plant/rhododendron-viscosum
https://fnps.org/plants/plant/rhus-aromatica
https://fnps.org/plants/plant/rhus-copallinum
https://fnp

,Availability,Considerations,Ethnobotany,Flower Color,Form,Fruit Color,Height,Life Span,Light,Moisture Tolerance,...,Width,Wildlife,Wildlife Notes,light_requirements,plant_aka,plant_link,plant_name,plant_scientific,suitable_zones,wildlife_list
0,"Native nurseries, FNPS plant sales, Friends",Brittle wood.,NaN,yellow,tree,brown,NaN,long-lived perennial,,,...,NaN,,Provides food and cover for birds and other wi...,[sun],"cinnecord, tamarindillo, thornless acacia",https://fnps.org/plants/plant/acacia-choriophylla,Acacia choriophylla,Fabaceae (Leguminosae),"10A,10B,11","[birds, animals, caterpillar, butterfly]"
1,"Native nurseries, FNPS plant sales, Friends","Fast growing, fairly weak hence somewhat prone...",NaN,"yellow,NA",tree,"green,brown",30-70 ft,long-lived perennial,,,...,20-40 ft,,"Attracts pollinators, particularly native bees...","[sun, part-shade]","box-elder, ash-leaf maple",https://fnps.org/plants/plant/acer-negundo,Acer negundo,Sapindaceae,"8A,8B,9A","[birds, animals, butterfly]"
2,"Big box stores, Quality nurseries, Native nurs...","Fast growing, fairly weak. Shallow, wide-sprea...",Sap once used as a source of sugar. Wood stri...,red,tree,red,45 to 50 ft,long-lived perennial,,,...,35 to 50 ft,,Birds and other wildlife consume seeds.\nLarva...,"[sun, part-shade]",red maple,https://fnps.org/plants/plant/acer-rubrum,Acer rubrum,Sapindaceae,"8A,8B,9A,9B,10A,10B","[birds, animals]"
3,"Native nurseries, FNPS plant sales",Its prolific root system is notorious for inva...,"Sap could be used for sugaring, but it's only ...","red,NA",tree,brown,75-100 ft,long-lived perennial,,,...,35-50 ft,,The buds of silver maple are important food fo...,"[sun, part-shade]",silver maple,https://fnps.org/plants/plant/acer-saccharinum,Acer saccharinum,Sapindaceae,"8A,8B","[birds, animals, caterpillar]"
4,"FNPS plant sales, Seed",None.,Has been used as a source of sap for sugar mak...,"green,NA",tree,brown,25-50 (85) ft,long-lived perennial,,,...,10-45 ft,,"Birds, bats and small mammals use as habitat a...","[part-shade, shade]",Florida maple,https://fnps.org/plants/plant/acer-saccharum-s...,Acer saccharum subsp. floridanum,Sapindaceae,"8A,8B,9A","[birds, animals, butterfly]"


In [102]:
florida_plant_df['Soil or other substrate']

2           Sand, clay, loam, organic material (muck)
6      Sand, loam, lime rock, organic material (muck)
7           Sand, clay, loam, organic material (muck)
8                               Sand, clay, lime rock
9                                          Sand, loam
10                                               Sand
12                                               Sand
13                                               Sand
14                                   Sand, clay, loam
17                                    Sand, lime rock
19                                               Sand
20                                               Sand
21                                         Sand, loam
24                                                NaN
27                                               Sand
31                                               Sand
32                                               Sand
34                                               Sand
35                          

In [96]:
florida_plant_df = pd.read_pickle('Florida_Native_Plants.pkl')
florida_plant_df['new_height'] =  fix_height(florida_plant_df.Height)
florida_plant_df[['height_min_ft','height_max_ft']] = pd.DataFrame(florida_plant_df.new_height.values.tolist())
florida_plant_df['9B_suitable'] = florida_plant_df.suitable_zones.apply(lambda x: '9B' in str(x))
florida_plant_df = florida_plant_df[florida_plant_df['9B_suitable']]
butterfly = florida_plant_df[florida_plant_df.wildlife_list.apply(lambda x: ('butterfly' in x)|('caterpillar' in x))]
print(butterfly.shape)

# filter for full-sun
# filter for sand soil
# filter for not deciduous
sun_sand_evergreen = butterfly[butterfly.light_requirements.apply(lambda x: 'sun' in x)]
sun_sand_evergreen = sun_sand_evergreen[sun_sand_evergreen['Soil or other substrate'].fillna('').apply(lambda x: 'sand' in x.lower())]
sun_sand_evergreen = sun_sand_evergreen[sun_sand_evergreen.Phenology.fillna('').apply(lambda x: 'deciduous' not in x.lower())]
print(sun_sand_evergreen.shape)


big_bushes = sun_sand_evergreen[sun_sand_evergreen.height_min_ft.replace('',6)>5]
big_bushes = big_bushes[big_bushes.height_max_ft.replace('',25)<=25]
print(big_bushes.shape)
big_bushes.head()

(264, 34)
(167, 34)
(31, 34)


,Availability,Considerations,Ethnobotany,Flower Color,Form,Fruit Color,Height,Life Span,Light,Moisture Tolerance,...,plant_aka,plant_link,plant_name,plant_scientific,suitable_zones,wildlife_list,new_height,height_min_ft,height_max_ft,9B_suitable
60,"Native nurseries, FNPS plant sales",NaN,NaN,"white,pink",shrub,black,6 to 8 ft,long-lived perennial,,,...,"tarflower, flyweed",https://fnps.org/plants/plant/bejaria-racemosa,Bejaria racemosa,Ericaceae,"8A,8B,9A,9B,10A,10B",[butterfly],"[6.0, 8.0]",6,8,True
109,Seed,NaN,NaN,"white,purple,lavender",vine,brown,NaN,short-lived perennial,,,...,spurred butterfly pea,https://fnps.org/plants/plant/centrosema-virgi...,Centrosema virginianum,Fabaceae (Leguminosae),"8A,8B,9A,9B,10A,10B,11",[caterpillar],"[, ]",,,True
124,"Quality nurseries, Native nurseries, FNPS plan...",NaN,The strong and heavy wood of this plant are al...,white,tree,brown,15 - 25 feet,long-lived perennial,,,...,fiddlewood,https://fnps.org/plants/plant/citharexylum-spi...,Citharexylum spinosum,Verbenaceae,"9B,10A,10B,11","[birds, animals, caterpillar, butterfly]","[15.0, 25.0]",15,25,True
203,Native nurseries,"Some say that it has a skunky odor, others can...",Fruits edible. Bark emulsions said to be a cur...,white,shrub,"red,black",10 to 20 (30) ft,long-lived perennial,,,...,white stopper,https://fnps.org/plants/plant/eugenia-axillaris,Eugenia axillaris,Myrtaceae,"9B,10A,10B,11","[birds, animals, butterfly]","[10.0, 20.0]",10,20,True
234,"Big box stores, Quality nurseries, Native nurs...",NaN,Toxic if eaten. Used in homeopathic medicine.,yellow,vine,NaN,To 25 ft,long-lived perennial,,,...,"yellow jesamine, Carolina jasamine",https://fnps.org/plants/plant/gelsemium-semper...,Gelsemium sempervirens,Gelsemiaceae,"8A,8B,9A,9B","[hummingbird, birds, caterpillar, butterfly]","[25.0, 25.0]",25,25,True


In [91]:
big_bushes[coi].to_csv('All_Florida_Zone9B_full_sun_big_bushes.csv', index=False)

In [98]:
flowers = sun_sand_evergreen[sun_sand_evergreen.height_min_ft.replace('',2)>=2]
flowers = flowers[flowers.height_max_ft.replace('',5)<=5]
print(flowers.shape)
flowers[coi].to_csv('All_Florida_Zone9B_full_sun_flowers.csv', index=False)

(65, 34)
